The <a href="https://data.bnf.fr">National Library of France</a> published its catalogue as Linked Open data.

This example shows how to use <a href="https://www.w3.org/TR/sparql11-query/">SPARQL</a> as a query language in Linked Open Data repositories.

### First of all, we init the SPARQLWrapper service with the SPARQL endpoint

In [8]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(
    "https://query.wikidata.org/sparql"
)
sparql.setReturnFormat(JSON)

### Let's write our first SPARQL sentence

We could be interested in the classes used in the repository

In [14]:
sparql.setQuery("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX rdarelationships: <http://rdvocab.info/RDARelationshipsWEMI/>
PREFIX rdagroup1elements: <http://rdvocab.info/Elements/>

SELECT ?author ?expression ?title ?edition ?placeOfPublication ?yearOfPublication ?langCode WHERE {
wd:Q2085725 wdt:P268 ?id
BIND(uri(concat(concat("http://data.bnf.fr/ark:/12148/cb", ?id),"#about")) as ?author)
SERVICE <http://data.bnf.fr/sparql> {
  ?expression <http://id.loc.gov/vocabulary/relators/aut> ?author .
  OPTIONAL {?expression dcterms:language ?langCode .}
  OPTIONAL {?expression dcterms:publisher ?edition .}
  ?manifestation rdarelationships:expressionManifested ?expression .
  ?manifestation dcterms:title ?title .
  ?manifestation dcterms:date ?yearOfPublication .
  OPTIONAL{ ?manifestation rdagroup1elements:placeOfPublication ?placeOfPublication .}
}
}limit 1000
"""
)

### Now we retrieve the results

In [15]:
try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:

        if "placeOfPublication" in r:
            print(r['placeOfPublication']['value'])
            
        if "yearOfPublication" in r:
            print(r['yearOfPublication']['value'])
        
        #print('Author: ', r['author']['value'], ' Work: ', r['work']['value'], 
        #      ' Title: ', r['workLabel']['value'], ' placeOfProduction: ', r['placeOfProduction']['value'],
        #      ' Year publication: ', r['yearOfPublication']['value'])
except Exception as e:
    print("Exception:")
    print(e)

London
1772
[S.l.]
1744
En Madrid
1749
[S.l.]
1744
Valladolid
2013
Nantes
1783
Londres
1826
[S.l.]
1744
Tours
1845
Paris, A. Boudet
1776
Amsterdam. - Leipzig
1752
En Madrid
1771
Madrid
1978
Madrid
1978
Madrid
1918
Madrid
1748
[S.l.]
1744
Matriti
1744
En Cadiz
1757
Madrid
1773
Madrid
1793
[S.l.]
1744
Madrid
1748
[S.l.]
1752
Londres
1826
Paris
1752
Madrid
1988
Paris
1776
Paris
1752
Madrid
1978
1991
1748


### Let's create a dataframe

In [18]:
data = []

for r in ret["results"]["bindings"]:

    placeOfPublication = ""
    if "placeOfPublication" in r:
        placeOfPublication = r['placeOfPublication']['value']

    yearOfPublication = ""
    if "yearOfPublication" in r:
        yearOfPublication = r['yearOfPublication']['value']

    edition = ""
    if "edition" in r:
        edition = r['edition']['value']    

    langCode = ''
    if "langCode" in r:
        langCode = r['langCode']['value']

    data.append({ "author" : r["author"]["value"], "work" : r["expression"]["value"], 
                  "title" : r["title"]["value"], "langCode" : langCode,
                  "edition" : edition, 
                  "yearOfPublication" : yearOfPublication, 
                  "placeOfProduction" : placeOfPublication})
print(data)

[{'author': 'http://data.bnf.fr/ark:/12148/cb119974653#about', 'work': 'http://data.bnf.fr/ark:/12148/cb45996910t#Expression', 'title': 'A voyage to South America. (George Juan and Antonio de Ulloa)', 'langCode': 'http://id.loc.gov/vocabulary/iso639-2/eng', 'edition': '', 'yearOfPublication': '1772', 'placeOfProduction': 'London'}, {'author': 'http://data.bnf.fr/ark:/12148/cb119974653#about', 'work': 'http://data.bnf.fr/ark:/12148/cb40756914b#Expression', 'title': 'Grundrifs von der Conceptions-bay... in Jahre 1744 aufgennomen...', 'langCode': 'http://id.loc.gov/vocabulary/iso639-2/ger', 'edition': '', 'yearOfPublication': '1744', 'placeOfProduction': '[S.l.]'}, {'author': 'http://data.bnf.fr/ark:/12148/cb119974653#about', 'work': 'http://data.bnf.fr/ark:/12148/cb30665853n#Expression', 'title': 'Dissertación histórica y geográphica sobre el meridiano de demarcación entre los dominios de España y Portugal, y los parages por donde passa en la América meridional, conforme á los tratados y

In [19]:
# check placeofproduction empty!!!
#data = [{"author" : x["author"]["value"], "work" : x["work"]["value"],
#        "title" : x["workLabel"]["value"], "placeOfProduction" : x["placeOfProduction"]["value"],
#        "yearOfPublication" : x["yearOfPublication"]["value"], 
#        "langCode" : x["langCode"]["value"]} for x in ret["results"]["bindings"]]

In [20]:
import pandas as pd

df = pd.DataFrame(data)

In [21]:
df.head(10)

,author,work,title,langCode,edition,yearOfPublication,placeOfProduction
0,http://data.bnf.fr/ark:/12148/cb119974653#about,http://data.bnf.fr/ark:/12148/cb45996910t#Expr...,A voyage to South America. (George Juan and An...,http://id.loc.gov/vocabulary/iso639-2/eng,,1772,London
1,http://data.bnf.fr/ark:/12148/cb119974653#about,http://data.bnf.fr/ark:/12148/cb40756914b#Expr...,Grundrifs von der Conceptions-bay... in Jahre ...,http://id.loc.gov/vocabulary/iso639-2/ger,,1744,[S.l.]
2,http://data.bnf.fr/ark:/12148/cb119974653#about,http://data.bnf.fr/ark:/12148/cb30665853n#Expr...,Dissertación histórica y geográphica sobre el ...,http://id.loc.gov/vocabulary/iso639-2/spa,,1749,En Madrid
3,http://data.bnf.fr/ark:/12148/cb119974653#about,http://data.bnf.fr/ark:/12148/cb40733276t#Expr...,Grundriss von dem Meerbusen und Hafen Valparay...,http://id.loc.gov/vocabulary/iso639-2/ger,,1744,[S.l.]
4,http://data.bnf.fr/ark:/12148/cb119974653#about,http://data.bnf.fr/ark:/12148/cb438889548#Expr...,Relacion historica del viage a la america meri...,http://id.loc.gov/vocabulary/iso639-2/spa,,2013,Valladolid
5,http://data.bnf.fr/ark:/12148/cb119974653#about,http://data.bnf.fr/ark:/12148/cb33994781x#Expr...,"Examen maritime théorique et pratique, ou trai...",http://id.loc.gov/vocabulary/iso639-2/fre,,1783,Nantes
6,http://data.bnf.fr/ark:/12148/cb119974653#about,http://data.bnf.fr/ark:/12148/cb30665847q#Expr...,"Noticias secretas de América, sobre el estado ...",http://id.loc.gov/vocabulary/iso639-2/spa,,1826,Londres
7,http://data.bnf.fr/ark:/12148/cb119974653#about,http://data.bnf.fr/ark:/12148/cb40758685h#Expr...,Plan sénographique (sic) de la cité des rois o...,http://id.loc.gov/vocabulary/iso639-2/fre,,1744,[S.l.]
8,http://data.bnf.fr/ark:/12148/cb119974653#about,http://data.bnf.fr/ark:/12148/cb31511015v#Expr...,Description de l'Amérique méridionale d'après ...,http://id.loc.gov/vocabulary/iso639-2/fre,,1845,Tours
9,http://data.bnf.fr/ark:/12148/cb119974653#about,http://data.bnf.fr/ark:/12148/cb32292716n#Expr...,Dissertation historique et géographique sur le...,http://id.loc.gov/vocabulary/iso639-2/fre,,1776,"Paris, A. Boudet"


In [22]:
df.describe()

,author,work,title,langCode,edition,yearOfPublication,placeOfProduction
count,32,32,32,32,32,32,32
unique,1,32,29,5,1,18,14
top,http://data.bnf.fr/ark:/12148/cb119974653#about,http://data.bnf.fr/ark:/12148/cb45996910t#Expr...,Plan sénographique (sic) de la cité des rois o...,http://id.loc.gov/vocabulary/iso639-2/spa,,1744,Madrid
freq,32,1,2,18,32,6,9


In [23]:
df.dtypes

author               object
work                 object
title                object
langCode             object
edition              object
yearOfPublication    object
placeOfProduction    object
dtype: object

In [24]:
sorted_df = df.sort_values(by='yearOfPublication')
print(sorted_df[["title","yearOfPublication"]].head(40))

                                                title yearOfPublication
1   Grundrifs von der Conceptions-bay... in Jahre ...              1744
3   Grundriss von dem Meerbusen und Hafen Valparay...              1744
21  Plan sénographique (sic) de la cité des rois o...              1744
17  Carta de la meridiana medida en el reyno de Qu...              1744
16  El puerto de El Callao, en el Mar Pacyfyco, od...              1744
7   Plan sénographique (sic) de la cité des rois o...              1744
15  Observaciones astronómicas y phísicas, hechas ...              1748
22  Relacion historica del viage a la América meri...              1748
31  Relacion historica del viage a la America meri...              1748
2   Dissertación histórica y geográphica sobre el ...              1749
10  Voyage historique de l'Amérique méridionale fa...              1752
28  Voyage historique de l'Amérique méridionale fa...              1752
25  [Illustrations de Voyage historique de l'Améri...           